# Gravitational Potential in Cartesian Coordinates

Calculates the paths of two objects that undergo a central potential of $U(r) = \frac{-Gm_1m_2}{r^2}$

Last Edited 4/29/2022 By: Matthew Freudenrich

Two objects of respective mass $m_1$ and $m_2$ have positions $\vec{r_1} = x_1 \hat{x} + y_1\hat{y}$ and $\vec{r_2} = x_2 \hat{x} + y_2\hat{y}$.

We define the vector going from $\vec{r_1}$ to $\vec{r_2}$ as $\vec{r}$ so that $\vec{r} = \vec{r_2} - \vec{r_1} = (x_2 - x_1)\hat{x} + (y_2 - y_2)\hat{y}$

Under a gravitational potential, the force on object one $\vec{F_1}$ is:

$\vec{F_1} = \frac{Gm_1m_2}{|\vec{r}|^2}\frac{\vec{r}}{|\vec{r}|}$

$\vec{F_1} = \frac{Gm_1m_2((x_2 - x_1)\hat{x} + (y_2 - y_2)\hat{y})}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$

And the force on object two $\vec{F_2}$ is:

$\vec{F_2} = -\frac{Gm_1m_2}{|\vec{r}|^2}\frac{\vec{r}}{|\vec{r}|}$

$\vec{F_2} = -\frac{Gm_1m_2((x_2 - x_1)\hat{x} + (y_2 - y_2)\hat{y})}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$

Applying Newton's second law, we see

$\vec{F_1} = m_1\vec{a_1} = m_1\ddot{x_1}\hat{x} + m_1\ddot{y_1}\hat{y}$

$\vec{F_2} = m_2\vec{a_2} = m_2\ddot{x_2}\hat{x} + m_2\ddot{y_2}\hat{y}$

This yields the differential equations:

$\ddot{x_1} = \frac{Gm_2(x_2-x_1)}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$

$\ddot{y_1} = \frac{Gm_2(y_2-y_1)}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$

$\ddot{x_2} = -\frac{Gm_1(x_2-x_1)}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$

$\ddot{y_2} = -\frac{Gm_1(y_2-y_1)}{((x_2 - x_1)^2 + (y_2-y_1))^{3/2}}$


We note:

$\ddot{x_2} = -\ddot{x_1}\frac{m_1}{m_2}$

and 

$\ddot{y_2} = -\ddot{y_1}\frac{m_1}{m_2}$

In [ ]:
%matplotlib inline
import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

class Gravity():
    """
    Class defining the behavior of the two masses as described in the markdown cell above
    Parameters
    ----------
    m1:float 
        mass of object 1
    m2:float
        mass of object 2
    sG:float
        strength of gravity
    
    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    
    """
    def __init__(self, m1=1., m2 = 1., sG = 1.):
        self.m1 = m1
        self.m2 = m2
        self.sG = sG
        
    def dy_dt(self, t, y):
        """
        This returns the right hand side of the diffeq 
        d/dt[x1, dx1/dt, y1, dy1/dt, x2, dx2/dt, y2, dy2/dt]
        
        Parameters
        ----------
        t:float
            time
        y:float
            an 8 component vector with y[0] = x1, y[1] = dx1/dt, y[2] = y1, y[3] = dy1/dt
                y[4] = x2, y[5] = dx2/dt, y[6] = y2, y[7] = dy2/dt
                
        Returns following expression:
        """
        return [y[1],(sG*m2*(y[4]-y[0]))/(((y[4]-y[0])**2 + (y[6]-y[2])**2)**(3./2.)),y[3],(sG*m2*(y[6]-y[2]))/(((y[4]-y[0])**2 + (y[6]-y[2])**2)**(3./2.)) ,y[5],(-sG*m1*(y[4]-y[0]))/(((y[4]-y[0])**2 + (y[6]-y[2])**2)**(3./2.)) ,y[7], (-sG*m1*(y[6]-y[2]))/(((y[4]-y[0])**2 + (y[6]-y[2])**2)**(3./2.))]
    
    
    
    def solve_ode(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = solution.y
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

#Labels for individual plot axis
x_vs_time_labels = (r'$t$', r'$x$')
x_dot_vs_time_labels = (r'$t$', r'$\dot{x}$')
y_vs_time_labels = (r'$t$', r'$y$')
y_dot_vs_time_labels = (r'$t$', r'$\dot{y}$')
y_vs_x_labels = (r'$x$', r'$y$')

Now we are ready to simulate the paths these objects take under a gravitational potential. 

In [ ]:
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 100.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

#Parameters

m1 = 1.
m2 = 1. 
sG = 1


#Initiate a system

g1 = Gravity(m1, m2, sG)

#Initial conditions
x1_0 = 0.
x1_dot_0 = 0.25
y1_0 = -1.
y1_dot_0 = 0.
x2_0 = 0.
x2_dot_0 = -0.25
y2_0 = 1.
y2_dot_0 = 0.

#Solving

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = g1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0)



Now we can plot:

In [ ]:
# start the plot!
fig = plt.figure(figsize=(15,5))
overall_title = 'Gravitation simulation ' + '\n' + \
                rf' $m1 = {m1:.2f},$' + \
                rf' $m2= {m2:.2f},$' + \
                rf' $sG = {sG:.2f},$' + \
                rf' $x1_0 = {x1_0:.2f},$' + \
                rf' $dx1/dt = {x1_dot_0:.2f},$' + \
                rf' $y1_0 = {y1_0:.2f},$' + \
                rf' $dy1/dt_0 = {y1_dot_0:.2f},$' + \
                rf' $x2_0 = {x2_0:.2f},$' + \
                rf' $dx2/dt_0 = {x2_dot_0:.2f},$' + \
                rf' $y2_0 = {y2_0:.2f},$' + \
                rf' $dy2/dt_0 = {y2_dot_0:.2f},$' + \
                '\n'  
fig.suptitle(overall_title, va='baseline')

#First plot: x1 vs time
ax_a = fig.add_subplot(1,5,1)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], x1[start : stop], 
            axis_labels=x_vs_time_labels, 
            color='blue',
            label=None, 
            title='$x1(t)$', 
            ax=ax_a)    

#Second plot: y1 vs time
ax_b = fig.add_subplot(1,5,2)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], y1[start : stop], 
            axis_labels=y_vs_time_labels, 
            color='blue',
            label=None, 
            title='$y1(t)$', 
            ax=ax_b) 

#Third plot: x2 vs time
ax_c = fig.add_subplot(1,5,3)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], x2[start : stop], 
            axis_labels=x_vs_time_labels, 
            color='red',
            label=None, 
            title='$x2(t)$', 
            ax=ax_c)    

#Fourth plot: y2 vs time
ax_d = fig.add_subplot(1,5,4)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(t_pts[start : stop], y2[start : stop], 
            axis_labels=y_vs_time_labels, 
            color='red',
            label=None, 
            title='$y2(t)$', 
            ax=ax_d)    

#Fifth plot: position of the system
ax_e = fig.add_subplot(1,5,5)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=y_vs_x_labels, 
            color='blue',
            label=None, 
            title='$Orbits$',
            ax=ax_e)
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=None, 
            color='red',
            label=None, 
            title=None,
            ax=ax_e)
fig.tight_layout()

Remembering the fact we found above:

$\ddot{x_2} = -\ddot{x_1}\frac{m_1}{m_2}$

and 

$\ddot{y_2} = -\ddot{y_1}\frac{m_1}{m_2}$

if $\frac{m_1}{m_2} \approx 0$ meaning $m_2 >> m_1$ then the accelerations $\ddot{x_2}$ and $\ddot{y_2}$ will be approximately zero. We can check this by plugging in the values:

m2 = 10000
m1 = 1

In [ ]:
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 100.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

#Parameters

m1 = 1.
m2 = 10000. 
sG = 0.01


#Initiate a system

g1 = Gravity(m1, m2, sG)

#Initial conditions
x1_0 = 0.
x1_dot_0 = 5.
y1_0 = -5.
y1_dot_0 = 0.
x2_0 = 0.
x2_dot_0 = 0.
y2_0 = 0.
y2_dot_0 = 0.

#Solving

x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = g1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0)



In [ ]:
#Plotting this system
fig = plt.figure(figsize=(5,5))
overall_title = 'Gravitation simulation ' + '\n' + \
                rf' $m1 = {m1:.2f},$' + \
                rf' $m2= {m2:.2f},$' + \
                rf' $sG = {sG:.2f},$' + \
                rf' $x1_0 = {x1_0:.2f},$' + \
                rf' $dx1/dt = {x1_dot_0:.2f},$' + \
                rf' $y1_0 = {y1_0:.2f},$' + \
                rf' $dy1/dt_0 = {y1_dot_0:.2f},$' + \
                rf' $x2_0 = {x2_0:.2f},$' + \
                rf' $dx2/dt_0 = {x2_dot_0:.2f},$' + \
                rf' $y2_0 = {y2_0:.2f},$' + \
                rf' $dy2/dt_0 = {y2_dot_0:.2f},$' + \
                '\n'  
fig.suptitle(overall_title, va='baseline')

ax_e = fig.add_subplot(1,1,1)

start, stop = start_stop_indices(t_pts, 0., 40.)    
plot_y_vs_x(x1[start : stop], y1[start : stop], 
            axis_labels=y_vs_x_labels, 
            color='blue',
            label=None, 
            title='$Orbits$',
            ax=ax_e)
plot_y_vs_x(x2[start : stop], y2[start : stop], 
            axis_labels=None, 
            color='red',
            label=None, 
            title=None,
            ax=ax_e)
fig.tight_layout()


We can clearly see the position of object 2 (the much more massive) in red is stationary, while object 1 orbits around it. 
